In [ ]:
#Reader: the dataset was too large to upload to Github. 
#We used the Expedia Challenge dataset, available as a free download from Kaggle

import pandas as pd
import gzip 
import numpy as np
%run map.py

with open('train.csv', 'rb') as fd:
    gzip_fd = gzip.GzipFile(fileobj=fd)
    data = pd.read_csv(gzip_fd)

In [349]:
ssr = .01 #subsampling rate
N = data.shape[0]

train = data.sample(n=int(ssr*N), axis=0, replace=False)
test = data.sample(n=int(.25*ssr*N), axis=0, replace=False)

train_users = train['user_id'].as_matrix()
train_clusters = train['hotel_cluster'].as_matrix()
train_ratings = train['is_booking'].as_matrix()

test_users  = test['user_id'].as_matrix()
test_clusters  = test['hotel_cluster'].as_matrix()
test_ratings = test['is_booking'].as_matrix()

del data

In [354]:
#Code here is by Kyle. For the theory, see Salakhutdinov et al., 2007: RBMs for collaborative filtering

import numpy as np
import itertools
import random
import copy
from scipy.sparse import csr_matrix

class Collaborative_RBM():
    
    def __init__(self,M,N,K,F,W,b_features,b_movies,V_all,H_all):
        self.M = M
        self.N = N
        self.K = K
        self.F = F
        self.W = W
        self.b_features = b_features
        self.b_movies = b_movies
        self.V_all = V_all
        self.H_all = H_all
   
    @staticmethod
    def softmax(arr):
        return np.exp(arr)/np.sum(np.exp(arr))

    @staticmethod
    def sigmoid(val):
        return 1/(1 + np.exp(-val))

    def p_v_given_h(self, h):
        vf = lambda l: self.b_movies[:,l] + np.dot(h, self.W[:,:,l].T)
        V_unnormalized = np.exp(np.array([vf(l) for l in range(self.K)])).T
        V = (V_unnormalized.T/np.sum(V_unnormalized, axis=1)).T
        return V
    
    def confidence_v_given_h(self, h):
        vf = lambda l: self.b_movies[:,l] + np.dot(h, self.W[:,:,l].T)
        return np.exp(np.array([vf(l) for l in range(self.K)])).T

    def p_h_given_V(self, V):
        vf = np.vectorize(lambda j : self.b_features[j] + np.sum(np.multiply(V, self.W[:,j])))
        return self.sigmoid(vf(np.arange(0,self.F)))    

    def vij_exp_data(self):
        deltaW_data = np.zeros((self.M,self.F,self.K))
        for u in range(self.N):
            V = np.vstack([np.zeros(K), np.eye(K)])[self.V_all[u].toarray()[0]]
            deltaW_data += (np.multiply.outer(V, self.p_h_given_V(V)).transpose(0,2,1))/self.N
        return deltaW_data

    def vij_exp_T(self,T):
        V_T = copy.copy(self.V_all)
        H_T = np.zeros((self.N,self.F))
        deltaW_data = np.zeros((self.M,self.F,self.K))

        vf1 = np.vectorize(lambda p : np.random.binomial(1,p))
        vf2 = lambda row : np.vstack([np.zeros(self.K), np.eye(self.K)])[np.random.choice(range(self.K), 1, list(row))]

        for epoch in range(T):
            V_Tp1 = copy.copy(V_T)
            H_Tp1 = copy.copy(H_T)

            #sample hidden states 
            for u in range(self.N):
                V = np.vstack([np.zeros(K), np.eye(K)])[V_T[u].toarray()[0]]
                H_Tp1[u] = vf1(self.p_h_given_V(V))

            #sample visible features
            for u in range(self.N):
                V_hot = self.p_v_given_h(H_Tp1[u])
                V_labels = np.zeros((1,self.M))
                for i in range(self.M):
                    V_labels[:,i] = vf2(V_hot[i])
                V_Tp1[u] = csr_matrix(V_labels)

            V_T = V_Tp1
            H_T = H_Tp1

            for u in range(self.N):
                V = np.vstack([np.zeros(K), np.eye(K)])[V_T[u].toarray()[0]]
                deltaW_data += (np.multiply.outer(V, H_T[u]).transpose(0,2,1))/(T*self.N)

            V_T = V_Tp1
            H_T = H_Tp1
        return deltaW_data
    
    def train(self, epochs, lr, gs_T):
        for epoch in range(epochs):            
            print(epoch)
            self.W += lr*(self.vij_exp_data() - self.vij_exp_T(gs_T))
            
    def pred(self):
        vf1 = np.vectorize(lambda p : np.random.binomial(1,p))
        vf2 = lambda row : np.vstack([np.zeros(self.K), np.eye(self.K)])[np.random.choice(range(self.K), 1, list(row))]
        
        H_pred = copy.copy(self.H_all)
        for u in range(self.N):
            V = np.vstack([np.zeros(K), np.eye(K)])[self.V_all[u].toarray()[0]]
            H_pred[u] = vf1(self.p_h_given_V(V))
                    
        V_pred = csr_matrix(np.zeros((N,M)))
        for u in range(self.N):
            V_confidences = self.confidence_v_given_h(H_pred[u]).T
            V_pred[u] = csr_matrix(V_confidences)
            
        return V_pred

In [386]:
#some helper functions

def compress(l):
    #recoverably sends an int list to an int list with max(compress(l)) = len(set(l)) - 1
    d = {}
    for i,j in enumerate(l):
        if j in d:
            d[j] += [i]
        else:
            d[j] = [i]
    arr = np.argsort(np.argsort(l))
    for j in d:
        for i in d[j]:
            arr[i] = arr[d[j][0]]
    return arr

def blowup(l, orig):
    #inverse of compress when passed original list orig
    return orig[np.argsort(orig)[l]]

def mapatk(k, probas):
    y_pred = (np.argsort(-probas, axis=1)[:, k:]).tolist()

    y_test_list = np.array([train_clusters]).T.tolist()
    return np.mean(mapk(y_test_list, y_pred, k))

def mapk(actual, predicted, k):
    return [apk(a,p,k) for a,p in zip(actual, predicted)]

array([12309, 35748,  5329, ..., 16027,  7249, 33404])

In [464]:
compressed_users = compress(train_users)
compressed_users_valid = compress(test_users)

N = compressed_users.shape[0]
Np = compressed_users_valid.shape[0]
M = max(train_clusters)+1
K = 1
F = 2
V_all_dense = np.zeros((N,M), dtype=np.int8)
for u in compressed_users:
    il = np.where(compressed_users==u)[0]
    V_all_dense[u][train_clusters[il]] = 1
V_all = csr_matrix(V_all_dense)

V_all_valid_dense = np.zeros((Np,M), dtype=np.int8)
for u in compressed_users_valid:
    il = np.where(compressed_users_valid==u)[0]
    V_all_dense[u][test_clusters[il]] = 1
V_all_valid = csr_matrix(V_all_valid_dense)

H_all = np.zeros((N,F))

W = np.random.normal(size = (M, F, K))
b_features = np.zeros((F))
b_movies = np.zeros((M,K))

model = Collaborative_RBM(K=K, N=N, M=M, F=F, H_all=H_all, V_all=V_all, W=W, b_features=b_features, b_movies=b_movies)

In [ ]:
model_valid = Collaborative_RBM(K=K, N=V_all_valid.shape[0], M=M, F=F, H_all=H_all, V_all=V_all_valid, W=model.W, b_features=b_features, b_movies=b_movies)
pred = model_valid.pred().toarray()

y_pred = np.flip((np.argsort(pred, axis=1))[:, -5:], axis=1).tolist()
y_test_list = np.array([test_clusters]).T.tolist()

np.mean(mapk(y_test_list, y_pred2, 5))